In [1]:
# 1. Extract key features ( contrast ,energy, corelation ) from the glcm --> do it using a dataset named MRI , which includes two other folders tumor and non-tumor containing 20 images in each folder , do what you can do
# 2. Compute the GLCM for a grayscale image with atleaast two different distances and angles ( eg. 1 pixel at 0 degree, 45 deg. , 90 dg., 135 deg. ) and normalize GLCM.



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def compute_glcm(image, distances=[1, 2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256):
    h, w = image.shape
    glcm_dict = {}

    for d in distances:
        for theta in angles:
            dx = int(d * np.cos(theta)) 
            dy = int(d * np.sin(theta)) 
            glcm = np.zeros((levels, levels), dtype=np.float64)

            for i in range(h - abs(dy)):
                for j in range(w - abs(dx)):
                    pixel_val = image[i, j]
                    neighbor_val = image[i + dy, j + dx]
                    glcm[pixel_val, neighbor_val] += 1
                    glcm[neighbor_val, pixel_val] += 1 

           
            glcm /= glcm.sum()
            glcm_dict[(d, theta)] = glcm

    return glcm_dict

image = cv2.imread('p5.jpg', cv2.IMREAD_GRAYSCALE)


glcm_matrices = compute_glcm(image, distances=[1, 2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])


for key, glcm in glcm_matrices.items():
    distance, angle = key
    print(f"GLCM for distance = {distance}, angle = {np.degrees(angle):.2f}°")
    print(glcm)
    print("-" * 50)


GLCM for distance = 1, angle = 0.00°
[[4.90031362e-04 3.18202183e-04 2.20832315e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.18202183e-04 1.25600766e-02 1.56173631e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.20832315e-04 1.56173631e-03 4.27205523e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.27280873e-05
  1.71829179e-05 3.05474096e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.71829179e-05
  2.29105572e-05 4.83667318e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.05474096e-05
  4.83667318e-05 1.04497597e-03]]
--------------------------------------------------
GLCM for distance = 1, angle = 45.00°
[[1.27156576e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.50413513e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.70860799e-02 ... 0.00000000e+00
  0.000000

In [10]:
import cv2
import numpy as np
import os
import pandas as pd

def compute_glcm(image, distances=[1, 2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256):
  
    h, w = image.shape
    glcm_dict = {}

    for d in distances:
        for theta in angles:
            dx = int(d * np.cos(theta))
            dy = int(d * np.sin(theta))
            glcm = np.zeros((levels, levels), dtype=np.float64)

            for i in range(h - abs(dy)):
                for j in range(w - abs(dx)):
                    pixel_val = image[i, j]
                    neighbor_val = image[i + dy, j + dx]
                    glcm[pixel_val, neighbor_val] += 1
                    glcm[neighbor_val, pixel_val] += 1  

            glcm /= glcm.sum()  
            glcm_dict[(d, theta)] = glcm

    return glcm_dict

def compute_features(glcm):
   
    levels = glcm.shape[0]
    i, j = np.indices((levels, levels))

    contrast = np.sum(glcm * (i - j) ** 2)
    energy = np.sum(glcm ** 2)
    
    mean_i = np.sum(i * glcm)
    mean_j = np.sum(j * glcm)
    std_i = np.sqrt(np.sum((i - mean_i) ** 2 * glcm))
    std_j = np.sqrt(np.sum((j - mean_j) ** 2 * glcm))

    correlation = np.sum(((i - mean_i) * (j - mean_j) * glcm) / (std_i * std_j + 1e-6))

    return contrast, energy, correlation

def process_dataset(dataset_path):
    
    categories = ["Tumor", "Non-Tumor"]
    results = []

    for category in categories:
        folder_path = os.path.join(dataset_path, category)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)

 
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue

    
            glcm_matrices = compute_glcm(image, distances=[1, 2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])


            for (d, theta), glcm in glcm_matrices.items():
                contrast, energy, correlation = compute_features(glcm)

               
                results.append({
                    "Image": filename,
                    "Category": category,
                    "Distance": d,
                    "Angle (Degrees)": np.degrees(theta),
                    "Contrast": contrast,
                    "Energy": energy,
                    "Correlation": correlation
                })

    return pd.DataFrame(results)


dataset_path = "MRI"


df = process_dataset(dataset_path)


df.to_csv("glcm_features_all_angles.csv", index=False)

print(df)


              Image   Category  Distance  Angle (Degrees)    Contrast  \
0    Tr-me_0020.jpg      Tumor         1              0.0   48.012032   
1    Tr-me_0020.jpg      Tumor         1             45.0    0.000000   
2    Tr-me_0020.jpg      Tumor         1             90.0   45.664131   
3    Tr-me_0020.jpg      Tumor         1            135.0    0.000000   
4    Tr-me_0020.jpg      Tumor         2              0.0  168.152880   
..              ...        ...       ...              ...         ...   
315  Tr-no_0025.jpg  Non-Tumor         1            135.0    0.000000   
316  Tr-no_0025.jpg  Non-Tumor         2              0.0  203.376177   
317  Tr-no_0025.jpg  Non-Tumor         2             45.0   98.977504   
318  Tr-no_0025.jpg  Non-Tumor         2             90.0  147.491824   
319  Tr-no_0025.jpg  Non-Tumor         2            135.0   95.125020   

       Energy  Correlation  
0    0.098580     0.990108  
1    0.110680     1.000000  
2    0.099540     0.990592  
3    0.